In [1]:
#處理所有.csv檔案將其轉換為長格式
import pandas as pd
import os
import re
from tqdm import tqdm

folder_path = r"D:\TTCIP\test"
output_folder = r"D:\TTCIP\test\1"
output_file_pattern = os.path.join(output_folder, "AR6_統計降尺度_日資料_臺灣_{}_{}_{}.csv")

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

file_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith(".csv") and filename != "output.csv"]
 
for i, file_path in enumerate(tqdm(file_paths, desc="Processing files")):
    filename = os.path.basename(file_path)
    match = re.search(r'AR6_統計降尺度_日資料_臺灣_([\w-]+)_([\w-]+)_(\d{4})', filename)
    if not match:
        print(f'Filename does not match regex: {filename}')
        continue

    var_type = match.group(1)
    model_name = match.group(2)
    year = match.group(3)

    try:
        df = pd.read_csv(file_path)
        df.fillna(-99.9, inplace=True)
        df.columns = ['LON', 'LAT'] + pd.to_datetime(df.columns[2:], format='%Y%m%d', errors='coerce').tolist()
        df_melt = df.melt(id_vars=["LON", "LAT"], var_name="DATE", value_name=f"{model_name}_{var_type}") 

        output_file = output_file_pattern.format(var_type, model_name, year)
        df_melt.to_csv(output_file, index=False)
        
        print(f"資料已成功寫入csv檔案：{output_file}")
        
        del df
        del df_melt

    except Exception as e:
        print(f'Error processing file: {filename}, error: {e}')


Processing files:  12%|█▎        | 1/8 [00:03<00:25,  3.66s/it]

資料已成功寫入csv檔案：D:\TTCIP\test\1\AR6_統計降尺度_日資料_臺灣_平均溫_ssp126_ACCESS-CM2_2015.csv


Processing files:  25%|██▌       | 2/8 [00:04<00:13,  2.27s/it]

資料已成功寫入csv檔案：D:\TTCIP\test\1\AR6_統計降尺度_日資料_臺灣_平均溫_ssp126_ACCESS-ESM1-5_2015.csv


Processing files:  38%|███▊      | 3/8 [00:06<00:09,  1.80s/it]

...

資料已成功寫入csv檔案：D:\TTCIP\test\1\AR6_統計降尺度_日資料_臺灣_最高溫_ssp126_ACCESS-ESM1-5_2015.csv


Processing files:  88%|████████▊ | 7/8 [00:11<00:01,  1.33s/it]

資料已成功寫入csv檔案：D:\TTCIP\test\1\AR6_統計降尺度_日資料_臺灣_降雨量_ssp126_ACCESS-CM2_2015.csv


Processing files: 100%|██████████| 8/8 [00:12<00:00,  1.56s/it]

資料已成功寫入csv檔案：D:\TTCIP\test\1\AR6_統計降尺度_日資料_臺灣_降雨量_ssp126_ACCESS-ESM1-5_2015.csv


In [2]:
#合併資料夾內.csv檔案
import os
import pandas as pd

# 設定主資料夾路徑
main_folder_path = r"D:\TTCIP\SSP126"

# 迴圈處理每個資料夾
for year in range(2015, 2101):
    folder_path = os.path.join(main_folder_path, str(year))
    
    # 獲取資料夾中所有的 CSV 檔案路徑
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

    # 初始化一個空的 DataFrame 用於存放最終結果
    combined_df = None

    # 處理每一個檔案
    for file_path in file_paths:
        # 讀取檔案並將其儲存為 DataFrame
        df = pd.read_csv(file_path)
    
        # 刪除日期欄位為空的行
        df = df.dropna(subset=['DATE'])
    
        # 如果這是第一個檔案，則將其指派給 combined_df
        if combined_df is None:
            combined_df = df
        else:
            # 對於後續的檔案，將它們與 combined_df 進行合併
            # 保留原始的欄位名稱
            # 使用 'inner' 合併來只保留在兩個 DataFrame 中都存在的行
            # 如果新的數據不為 NaN，則用新的數據更新現有的數據
            combined_df = combined_df.combine_first(df)

    
    # 在合併後的 DataFrame 中，將第一個欄位的空白標題填入 "number"
    if combined_df.columns[0] == '':
        combined_df.rename(columns={'': 'number'}, inplace=True)

    # 將所有的空值替換為-99.9
    combined_df.fillna(-99.9, inplace=True)
    
    # 將合併後的 DataFrame 轉存為 Excel 文件
    # 將列名轉換為小寫
    combined_df.columns = [column.lower() for column in combined_df.columns]
    combined_df.to_excel(os.path.join(folder_path, f"{year}_data.xlsx"), encoding='utf-8')
    



   ACCESS-CM2_平均溫_ssp126  ACCESS-CM2_最低溫_ssp126  ACCESS-CM2_最高溫_ssp126  \
0                  -99.9                  -99.9                  -99.9   
1                  -99.9                  -99.9                  -99.9   
2                  -99.9                  -99.9                  -99.9   
3                  -99.9                  -99.9                  -99.9   
4                  -99.9                  -99.9                  -99.9   

   ACCESS-CM2_降雨量_ssp126  ACCESS-ESM1-5_平均溫_ssp126  ACCESS-ESM1-5_最低溫_ssp126  \
0                  -99.9                     -99.9                     -99.9   
1                  -99.9                     -99.9                     -99.9   
2                  -99.9                     -99.9                     -99.9   
3                  -99.9                     -99.9                     -99.9   
4                  -99.9                     -99.9                     -99.9   

   ACCESS-ESM1-5_最高溫_ssp126  ACCESS-ESM1-5_降雨量_ssp126        DATE    LAT  

d:\ANACONDA\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [ ]:
#創建資料庫表格，寫入資料庫
import psycopg2

database = "Rawdata_NCDR"

conn = psycopg2.connect(database=database, user="postgres", password="1!qaz2@wsx", host="140.134.51.64", port="5432")

cursor = conn.cursor()
create_table_query = '''
    CREATE TABLE ncdr_ssp_126 (
        number INTEGER,
        lon FLOAT,
        lat FLOAT,
        date DATE,
        inm_cm4_8_最低溫_ssp126 float,kace_1_0_g_最高溫_ssp126 float,cmcc_esm2_降雨量_ssp126 float,mpi_esm1_2_hr_降雨量_ssp126 float,inm_cm5_0_平均溫_ssp126 float,canesm5_平均溫_ssp126 float,miroc6_降雨量_ssp126 float,iitm_esm_降雨量_ssp126 float,nor esm2_lm_最低溫_ssp126 float,access_esm1_5_最高溫_ssp126 float,bcc_csm2_mr_最高溫_ssp126 float,tai esm1_降雨量_ssp126 float,awi_cm_1_1_mr_最低溫_ssp126 float,inm_cm5_0_最高溫_ssp126 float,ipsl_cm6a_lr_平均溫_ssp126 float,tai esm1_最低溫_ssp126 float,fg oals_g3_最高溫_ssp126 float,access_esm1_5_平均溫_ssp126 float,gfdl_esm4_降雨量_ssp126 float,ec_earth3_veg_最高溫_ssp126 float,ec_earth3_veg_lr_降雨量_ssp126 float,ec_earth3_最高溫_ssp126 float,ipsl_cm6a_lr_降雨量_ssp126 float,mri_esm2_0_最低溫_ssp126 float,nor esm2_lm_平均溫_ssp126 float,access_cm2_最高溫_ssp126 float,mri_esm2_0_最高溫_ssp126 float,gfdl_esm4_平均溫_ssp126 float,nesm3_平均溫_ssp126 float,iitm_esm_平均溫_ssp126 float,gfdl_esm4_最高溫_ssp126 float,kiost_esm_最低溫_ssp126 float,mpi_esm1_2_lr_平均溫_ssp126 float,fg oals_g3_降雨量_ssp126 float,ec_earth3_veg_lr_平均溫_ssp126 float,access_cm2_最低溫_ssp126 float,mpi_esm1_2_hr_最低溫_ssp126 float,nor esm2_lm_降雨量_ssp126 float,mpi_esm1_2_hr_平均溫_ssp126 float,ipsl_cm6a_lr_最高溫_ssp126 float,nesm3_最低溫_ssp126 float,fg oals_g3_最低溫_ssp126 float,gfdl_esm4_最低溫_ssp126 float,nor esm2_mm_最低溫_ssp126 float,canesm5_最高溫_ssp126 float,access_esm1_5_最低溫_ssp126 float,bcc_csm2_mr_最低溫_ssp126 float,access_esm1_5_降雨量_ssp126 float,nesm3_降雨量_ssp126 float,nor esm2_mm_降雨量_ssp126 float,mpi_esm1_2_lr_最低溫_ssp126 float,bcc_csm2_mr_平均溫_ssp126 float,inm_cm4_8_最高溫_ssp126 float,ec_earth3_veg_平均溫_ssp126 float,cmcc_esm2_平均溫_ssp126 float,tai esm1_平均溫_ssp126 float,kace_1_0_g_最低溫_ssp126 float,awi_cm_1_1_mr_最高溫_ssp126 float,nor esm2_lm_最高溫_ssp126 float,kace_1_0_g_平均溫_ssp126 float,kiost_esm_降雨量_ssp126 float,miroc6_最低溫_ssp126 float,inm_cm5_0_最低溫_ssp126 float,miroc6_最高溫_ssp126 float,canesm5_降雨量_ssp126 float,mpi_esm1_2_lr_降雨量_ssp126 float,bcc_csm2_mr_降雨量_ssp126 float,ec_earth3_最低溫_ssp126 float,miroc6_平均溫_ssp126 float,inm_cm4_8_降雨量_ssp126 float,ipsl_cm5a2_inca_降雨量_ssp126 float,kiost_esm_平均溫_ssp126 float,ec_earth3_veg_lr_最高溫_ssp126 float,iitm_esm_最低溫_ssp126 float,ec_earth3_veg_lr_最低溫_ssp126 float,fg oals_g3_平均溫_ssp126 float,awi_cm_1_1_mr_平均溫_ssp126 float,mri_esm2_0_平均溫_ssp126 float,mpi_esm1_2_lr_最高溫_ssp126 float,awi_cm_1_1_mr_降雨量_ssp126 float,inm_cm5_0_降雨量_ssp126 float,access_cm2_降雨量_ssp126 float,ec_earth3_veg_最低溫_ssp126 float,canesm5_最低溫_ssp126 float,kiost_esm_最高溫_ssp126 float,access_cm2_平均溫_ssp126 float,ec_earth3_veg_降雨量_ssp126 float,cmcc_esm2_最高溫_ssp126 float,cesm2_waccm_降雨量_ssp126 float,iitm_esm_最高溫_ssp126 float,mpi_esm1_2_hr_最高溫_ssp126 float,mri_esm2_0_降雨量_ssp126 float,ec_earth3_平均溫_ssp126 float,ipsl_cm6a_lr_最低溫_ssp126 float,nesm3_最高溫_ssp126 float,nor esm2_mm_平均溫_ssp126 float,ec_earth3_降雨量_ssp126 float,inm_cm4_8_平均溫_ssp126 float,kace_1_0_g_降雨量_ssp126 float,cmcc_cm2_sr5_降雨量_ssp126 float,cmcc_esm2_最低溫_ssp126 float)
'''

# 執行建立表格的指令
cursor.execute(create_table_query)
conn.commit()

# 關閉資料庫連接
cursor.close()
conn.close()